## Amdahl's Law 

Improving a portion P of a computation by factor s results in an overall speedup of

<img src="./images/al1.png" width="256" title="Amdahl's Law Equation" />

* _p_ is the proportion of execution time that benefits from improved resources, i.e. the parallel part
* _(1-p)_ is the portion that does not benefit; i.e. the serial part
* _s_ is the speedup of the optimized part
* _S<sub>latency</sub>(s)_ theoretical speedup of the whole task

Paraphrased: speedup limited to fraction improved

<img src="./images/al2.png" width="512" title="Amdahl's Law Graph" />

### Visualizing Amdahl's Law



In [6]:
library(ggplot2)

# Let's assume we have a 4 socket x 16 core machine.  So up to 64 cores.
cores = seq(1:1:64)

# and a parallel fraction of 99%
p = .99

al = 1/((1-p)+p/cores)

gplot()

 [1]  1.000000  1.980198  2.941176  3.883495  4.807692  5.714286  6.603774
 [8]  7.476636  8.333333  9.174312 10.000000 10.810811 11.607143 12.389381
[15] 13.157895 13.913043 14.655172 15.384615 16.101695 16.806723 17.500000
[22] 18.181818 18.852459 19.512195 20.161290 20.800000 21.428571 22.047244
[29] 22.656250 23.255814 23.846154 24.427481 25.000000 25.563910 26.119403
[36] 26.666667 27.205882 27.737226 28.260870 28.776978 29.285714 29.787234
[43] 30.281690 30.769231 31.250000 31.724138 32.191781 32.653061 33.108108
[50] 33.557047 34.000000 34.437086 34.868421 35.294118 35.714286 36.129032
[57] 36.538462 36.942675 37.341772 37.735849 38.125000 38.509317 38.888889
[64] 39.263804


### Infering parallelism from Amdahl's Law